In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd '/content/drive/My Drive/Colab Notebooks/yolov2'

/content/drive/My Drive/Colab Notebooks/yolov2


In [0]:
import torch
import args
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.autograd import Variable
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter

from dataset import VOCDataset
from yolov2 import YOLOv2
from utils import draw_boxes, get_detection_result

In [0]:
import argparse
parser = argparse.ArgumentParser(description="A YOLOv2 detector with Darknet19 backbone!")
args_ = parser.parse_args(args=[])

In [0]:
# define and load YOLOv2
net = YOLOv2(args_)
net.load_weight("./darknet19_448.conv.23")
net.train()

YOLOv2(
  (conv1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.1, inplace=True)
      (4): Conv2

In [0]:
# %cd /content/drive/'My Drive'/'Colab Notebooks/dataset'
# !tar -xvf VOCtrainval_11-May-2012.tar
!ls ../dataset/VOCdevkit/

VOC2012


In [0]:
!pwd

/content/drive/My Drive/Colab Notebooks/yolov2


In [0]:
# define optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-6, momentum=0.9, weight_decay=5e-4)

training_set = VOCDataset("../dataset/VOCdevkit/", "2012", "train", image_size=416)
dataloader = DataLoader(training_set, shuffle= True, batch_size=2)

In [0]:
def train():

    writer = SummaryWriter()
    writer.add_graph(net, torch.rand(4, 3, 416, 416))

    for batch_idx, (images, labels) in enumerate(dataloader):
        print("===============> step: {} <=================".format(batch_idx))
        images, labels = Variable(images, requires_grad = True), labels

        optimizer.zero_grad()
        output = net.forward(images)
        print("output shape: ", output.shape)
        loss, loss_coord, loss_conf, loss_cls = net.loss(output, labels)
        loss.backward()
        optimizer.step()
        

        writer.add_scalar('Train/Total_loss', loss, batch_idx)
        writer.add_scalar('Train/Coordination_loss', loss_coord, batch_idx)
        writer.add_scalar('Train/Confidence_loss', loss_conf, batch_idx)
        writer.add_scalar('Train/Class_loss', loss_cls, batch_idx)

        if batch_idx % 10 == 0:
            boxes = get_detection_result(output, conf_thres=.8, nms_thres=0.4)

            # draw detected boxes and save sample
            im = images[0].data.numpy().astype('uint8')
            im = im.transpose(1,2,0)
            im = im.copy()

            color_red = (0, 0, 255)
            color_green = (0, 255, 0)
            im = draw_boxes(im, labels[0], color=color_green)
            im = draw_boxes(im, boxes[0], color = color_red)
            cv2.imwrite("result/result_iter_{}.jpg".format(batch_idx), im)

    writer.close()

In [0]:
train()

/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:50: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.view(B, C, int(H / hs), hs, int(W / ws), ws).transpose(3, 4).contiguous()
/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:51: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.view(B, C, int(H / hs * W / ws), hs * ws).transpose(2, 3).contiguous()
/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:52: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of

Streaming output truncated to the last 5000 lines.
         [ 6.3648,  8.5411],
         [ 6.5256,  9.5514],
         [ 6.3569, 10.5095],
         [ 6.4868, 11.5675],
         [ 6.5088, 12.4687]]], grad_fn=<SelectBackward>)
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
keep:  tensor([0])
object:  tensor([219.6480, 259.5840, 271.2320, 212.9920,  11.0000], dtype=torch.float64)
object:  tensor([226.3040, 150.8693, 173.8880,  59.9040,  11.0000], dtype=torch.float64)
object:  tensor([ 48.4879,  16.5968,  91.5308, 117.6753,  17.0000],
       grad_fn=<SelectBackward>)
===============> step: 1 <=================
output shape:  torch.Size([2, 125, 13, 13])
true_box_conf:  tensor([0.1017, 0.2262])
best iou shape:  torch.Size([2, 845])
confidence mask: #OBJ 2.0, #NO_OBJ 1679.0, #TOTAL: 1681.0, #OBJ + #NO_OBJ == #TOTAL? True
 =====> nb_coord_box:  tensor(2.)
 =====> nb_conf_box:  tensor(1681.)

/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:50: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.view(B, C, int(H / hs), hs, int(W / ws), ws).transpose(3, 4).contiguous()
/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:51: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x.view(B, C, int(H / hs * W / ws), hs * ws).transpose(2, 3).contiguous()
/content/drive/My Drive/Colab Notebooks/yolov2/yolov2.py:52: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of

Streaming output truncated to the last 5000 lines.
        [ 5.6371,  3.2978],
        [ 2.7998,  7.8685]], grad_fn=<IndexBackward>)
true  coord:  tensor([[ 3.4840, 11.1627],
        [ 0.4940,  1.1787],
        [ 3.3800,  5.4773],
        [ 4.3680, 12.8267],
        [ 1.4300,  2.4613],
        [ 1.4560,  2.4267],
        [ 1.2740,  4.0907],
        [ 0.8320,  0.6953],
        [ 5.3820,  2.9023],
        [ 6.0840,  8.0116],
        [ 0.5720,  0.6047],
        [ 0.9620,  0.8465],
        [ 1.8460,  8.7372],
        [ 2.0540,  1.3605],
        [ 6.2140,  9.7047],
        [ 3.6140,  1.6326]])
Loss xy:  tensor(44.3018, grad_fn=<DivBackward0>)
Loss wh:  tensor(16.1729, grad_fn=<DivBackward0>)
loss class:  tensor(3.1454, grad_fn=<MulBackward0>)
loss conf:  tensor(0.3386, grad_fn=<DivBackward0>)
total loss:  tensor(63.9587, grad_fn=<AddBackward0>)
===============> step: 19 <=================
output shape:  torch.Size([2, 125, 13, 13])
true_box_conf:  tensor([0.2373])
best iou shape:  torch.Siz